In [1]:
from slstm import sLSTM
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import numpy as np
import time
from torch.optim.lr_scheduler import StepLR
def trainer(model, epochs, train_loader, val_loader, loss_fn, optim, scheduler):
    train_losses = []
    val_losses = []
    best_train_loss = float('inf')
    best_val_loss = float('inf')
    best_train_epoch = 0
    best_val_epoch = 0

    for epoch in range(1, epochs+1):
        # Training
        model.train()
        num_batches = len(train_loader)
        total_train_loss = 0
        for x, y in train_loader:
            x, y = x.to('cuda'), y.to('cuda')
            output = model(x)
            loss = loss_fn(output, y)
            optim.zero_grad()
            loss.backward()
            optim.step()
            total_train_loss += loss.item()
        scheduler.step()
        avg_train_loss = np.sqrt(total_train_loss / num_batches)
        train_losses.append(avg_train_loss)
        
        if avg_train_loss < best_train_loss:
            best_train_loss = avg_train_loss
            best_train_epoch = epoch

        # Validation
        model.eval()
        num_val_batches = len(val_loader)
        total_val_loss = 0
        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to('cuda'), y.to('cuda')
                output = model(x)
                loss = loss_fn(output, y)
                total_val_loss += loss.item()
        
        avg_val_loss = np.sqrt(total_val_loss / num_val_batches)
        val_losses.append(avg_val_loss)
        
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            best_val_epoch = epoch

        print(f'Epoch {epoch} Train RMSE Loss: {avg_train_loss:.4f}, Val RMSE Loss: {avg_val_loss:.4f}')

        # Save results to a .txt file
        with open('test_3_round_2_training_results.txt', 'w') as f:
            f.write(f'Best Training RMSE Loss: {best_train_loss:.4f} at epoch {best_train_epoch}\n')
            f.write(f'Best Validation RMSE Loss: {best_val_loss:.4f} at epoch {best_val_epoch}\n')
            f.write('\nEpoch-wise losses:\n')
            for epoch, (train_loss, val_loss) in enumerate(zip(train_losses, val_losses), 1):
                f.write(f'Epoch {epoch}: Train RMSE = {train_loss:.4f}, Val RMSE = {val_loss:.4f}\n')

    return train_losses, val_losses, best_train_loss, best_val_loss, best_train_epoch, best_val_epoch


train = torch.load('../../../../data/cleaned/train.pt')
val = torch.load('../../../../data/cleaned/val.pt')
batch_size = 32
train_loader = DataLoader(train, batch_size, shuffle=True, drop_last=True)
val_loader = DataLoader(val, batch_size=32, shuffle=True, drop_last=True)
model = sLSTM()
model.to('cuda')
loss_fn = torch.nn.MSELoss()
optim = torch.optim.Adam(params=model.parameters(), lr=0.001)
scheduler = StepLR(optim, step_size=30)
start = time.time()
losses = trainer(model, 1000, train_loader, val_loader, loss_fn, optim, scheduler)
end = time.time()

print(f'\nTotal training time on 100 epochs: {end-start}')

Epoch 1 Train RMSE Loss: 288.2275, Val RMSE Loss: 134.0161
Epoch 2 Train RMSE Loss: 283.3244, Val RMSE Loss: 143.8724
Epoch 3 Train RMSE Loss: 283.9671, Val RMSE Loss: 133.1550
Epoch 4 Train RMSE Loss: 281.5514, Val RMSE Loss: 137.0636
Epoch 5 Train RMSE Loss: 280.1328, Val RMSE Loss: 136.4367
Epoch 6 Train RMSE Loss: 278.4559, Val RMSE Loss: 113.5969
Epoch 7 Train RMSE Loss: 276.3810, Val RMSE Loss: 92.4231
Epoch 8 Train RMSE Loss: 276.6132, Val RMSE Loss: 152.5335
Epoch 9 Train RMSE Loss: 274.2767, Val RMSE Loss: 179.6255
Epoch 10 Train RMSE Loss: 270.4975, Val RMSE Loss: 143.2747
Epoch 11 Train RMSE Loss: 269.8523, Val RMSE Loss: 150.1328
Epoch 12 Train RMSE Loss: 263.4765, Val RMSE Loss: 134.5475
Epoch 13 Train RMSE Loss: 262.9734, Val RMSE Loss: 136.6680
Epoch 14 Train RMSE Loss: 274.2720, Val RMSE Loss: 149.4283
Epoch 15 Train RMSE Loss: 268.3254, Val RMSE Loss: 131.9094
Epoch 16 Train RMSE Loss: 270.0156, Val RMSE Loss: 138.0898
Epoch 17 Train RMSE Loss: 268.4041, Val RMSE Loss: